In [1]:
# Get pipeline run id
pipeline_run_id = ''

StatementMeta(synspdevposei, 33, 1, Finished, Available)

In [5]:
import os
import datetime

# Get keyvault linked service name
keyvaultlsname = 'Ls_KeyVault_01'

token_library = sc._jvm.com.microsoft.azure.synapse.tokenlibrary.TokenLibrary

# Primary storage info 
storage_account_url = token_library.getSecretWithLS(keyvaultlsname, "datalakeurl").replace("https://", '')
container_name = 'saveddata' # fill in your container name 
relative_path = '' # fill in your relative folder path 
file_name = f"{pipeline_run_id}.parquet"
adls_path = 'abfss://%s@%s/%s' % (container_name, storage_account_url, relative_path) 
print('Primary storage account path: ' + adls_path) 
loaded_on = datetime.datetime.now()
base_path = os.path.join(adls_path, file_name)
print('Base path is: ' + base_path)
df = spark.read.parquet(base_path)

df.printSchema()
df.show()

StatementMeta(synspdevposei, 33, 5, Finished, Available)

Primary storage account path: abfss://saveddata@mdwdopsstdevposei.dfs.core.usgovcloudapi.net/
Base path is: abfss://saveddata@mdwdopsstdevposei.dfs.core.usgovcloudapi.net/0078a5a8-c9f8-4a69-88f3-70614def99e3.parquet
root
 |-- dim_date_id: string (nullable = true)
 |-- date: string (nullable = true)
 |-- day: string (nullable = true)
 |-- day_suffix: string (nullable = true)
 |-- week_day: string (nullable = true)
 |-- week_day_name: string (nullable = true)
 |-- DOW_in_month: string (nullable = true)
 |-- day_of_year: string (nullable = true)
 |-- week_of_month: string (nullable = true)
 |-- week_of_year: string (nullable = true)
 |-- ISO_week_of_year: string (nullable = true)
 |-- month: string (nullable = true)
 |-- month_name: string (nullable = true)
 |-- quarter: string (nullable = true)
 |-- quarter_name: string (nullable = true)
 |-- year: string (nullable = true)
 |-- MMYYYY: string (nullable = true)
 |-- month_year: string (nullable = true)
 |-- first_day_of_month: string (nul

In [7]:
count = df.count()
data = [[loaded_on, file_name, pipeline_run_id, df.count()]]
 
# specify column names
columns = ['date', 'filename', 'pipeline_run_id', 'count']
 
# creating a dataframe from the lists of data
dataframe = spark.createDataFrame(data, columns)

dataframe.show()

StatementMeta(synspdevposei, 33, 7, Finished, Available)

+--------------------+--------------------+--------------------+-----+
|                date|            filename|     pipeline_run_id|count|
+--------------------+--------------------+--------------------+-----+
|2022-12-09 17:03:...|0078a5a8-c9f8-4a6...|0078a5a8-c9f8-4a6...|   99|
+--------------------+--------------------+--------------------+-----+



In [8]:
dataframe.write.mode('append').parquet(os.path.join(adls_path, 'interim.count'))

StatementMeta(synspdevposei, 33, 8, Finished, Available)